In [4]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [5]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [11]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 20
n_iter = 300000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [13]:
# import impl.RNN as nn
# net = nn.RNN(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.LSTM as nn
# net = nn.LSTM(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

import impl.GRU as nn
net = nn.GRU(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU2 as nn
# net = nn.GRU2(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU3 as nn
# net = nn.GRU3(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# # GRU4 goes to inf because the res-conn is gone.
# import impl.GRU4 as nn
# net = nn.GRU4(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [ ]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step, # This is the width of the network
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.2577
in thy lopary ohe litla Oie the aryl cintehreno tloctes frint and uhgeas aof Aror jhe hJorst be wock


Iter-2000 loss: 2.5039
otud of and panne earon's aigeowon ealad Wst lert iunjition, Japan Billd on ceanno, 4hy the tourly J




In [ ]:
=========================================================================
Iter-299000 loss: 0.0914
=========================================================================
s. Japan's history. From the 12th century until 1868, Japan's n7meamean Laas pan hs, Japan was men M
=========================================================================


=========================================================================
Iter-300000 loss: 0.1100
=========================================================================
apan was inhabited as early as the Upper Paleolithic period. The first are Honsh mict of inter of in
=========================================================================


Out[10]:
<impl.LSTM.LSTM at 0x7f51924f1940>

In [ ]:
=========================================================================
Iter-299000 loss: 0.1178
=========================================================================
s. Japan is a developed country wino avinsm itso. The foulslint of is in the namiort tites Toky byde
=========================================================================


=========================================================================
Iter-300000 loss: 0.0913
=========================================================================
apan was ruled in the name of the Emperor as a d bevere Hf Hiros. Japan is athe world's third-s. Jap
=========================================================================


Out[8]:
<impl.RNN.RNN at 0x7f51924f1630>